# Dimensión fecha

### Librerías necesarias

In [1]:
from datetime import date
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import holidays
import yaml

### Módulo de extracción

In [2]:
# Creación del dataframe
dim_fecha = pd.DataFrame({"date": pd.date_range(start='1/1/2005', end='1/1/2006', freq='D')})
#dim_fecha.head()
print(dim_fecha)

          date
0   2005-01-01
1   2005-01-02
2   2005-01-03
3   2005-01-04
4   2005-01-05
..         ...
361 2005-12-28
362 2005-12-29
363 2005-12-30
364 2005-12-31
365 2006-01-01

[366 rows x 1 columns]


### Módulo de transformación

In [3]:
dim_fecha["day"] = dim_fecha["date"].dt.day
dim_fecha["month"] = dim_fecha["date"].dt.month
dim_fecha["year"] = dim_fecha["date"].dt.year
dim_fecha["weekday"] = dim_fecha["date"].dt.weekday
dim_fecha["quarter"] = dim_fecha["date"].dt.quarter

print(dim_fecha)

          date  day  month  year  weekday  quarter
0   2005-01-01    1      1  2005        5        1
1   2005-01-02    2      1  2005        6        1
2   2005-01-03    3      1  2005        0        1
3   2005-01-04    4      1  2005        1        1
4   2005-01-05    5      1  2005        2        1
..         ...  ...    ...   ...      ...      ...
361 2005-12-28   28     12  2005        2        4
362 2005-12-29   29     12  2005        3        4
363 2005-12-30   30     12  2005        4        4
364 2005-12-31   31     12  2005        5        4
365 2006-01-01    1      1  2006        6        1

[366 rows x 6 columns]


In [4]:
dim_fecha["day_of_year"] = dim_fecha["date"].dt.day_of_year
dim_fecha["day_of_month"] = dim_fecha["date"].dt.days_in_month
dim_fecha["month_str"] = dim_fecha["date"].dt.month_name()
dim_fecha["day_str"] = dim_fecha["date"].dt.day_name()
dim_fecha["date_str"] = dim_fecha["date"].dt.strftime("%d/%m/%Y")
print(dim_fecha)

          date  day  month  year  weekday  quarter  day_of_year  day_of_month  \
0   2005-01-01    1      1  2005        5        1            1            31   
1   2005-01-02    2      1  2005        6        1            2            31   
2   2005-01-03    3      1  2005        0        1            3            31   
3   2005-01-04    4      1  2005        1        1            4            31   
4   2005-01-05    5      1  2005        2        1            5            31   
..         ...  ...    ...   ...      ...      ...          ...           ...   
361 2005-12-28   28     12  2005        2        4          362            31   
362 2005-12-29   29     12  2005        3        4          363            31   
363 2005-12-30   30     12  2005        4        4          364            31   
364 2005-12-31   31     12  2005        5        4          365            31   
365 2006-01-01    1      1  2006        6        1            1            31   

    month_str    day_str   

In [5]:
co_holidays = holidays.CO(language="es")
dim_fecha["is_Holiday"] = dim_fecha["date"].apply(lambda x:  x in co_holidays)
dim_fecha["holiday"] = dim_fecha["date"].apply(lambda x: co_holidays.get(x))
dim_fecha["saved"] = date.today()
dim_fecha["weekend"] = dim_fecha["weekday"].apply(lambda x: x>4)
dim_fecha.head()

,date,day,month,year,weekday,quarter,day_of_year,day_of_month,month_str,day_str,date_str,is_Holiday,holiday,saved,weekend
0,2005-01-01,1,1,2005,5,1,1,31,January,Saturday,01/01/2005,True,Año Nuevo,2024-10-24,True
1,2005-01-02,2,1,2005,6,1,2,31,January,Sunday,02/01/2005,False,None,2024-10-24,True
2,2005-01-03,3,1,2005,0,1,3,31,January,Monday,03/01/2005,False,None,2024-10-24,False
3,2005-01-04,4,1,2005,1,1,4,31,January,Tuesday,04/01/2005,False,None,2024-10-24,False
4,2005-01-05,5,1,2005,2,1,5,31,January,Wednesday,05/01/2005,False,None,2024-10-24,False


### Módulo de carga a la bodega de datos

In [6]:
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")
#print(url_oltp)
#print(url_olap)
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

In [7]:
try: # Realizar una consulta sencilla para verificar la conexión
    # Ejecutar una consulta para obtener el nombre de las tablas
    tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';", olap_conn)
    #print(tables)
    print("Conexión establecida y consulta realizada con éxito.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

Conexión establecida y consulta realizada con éxito.


In [8]:
dim_fecha.to_sql('dim_fecha', olap_conn, if_exists='replace',index_label='key_dim_fecha')

366